In [1]:
%matplotlib inline

In [2]:
import numpy as np
import math
from numpy import expand_dims
from scipy.integrate import odeint,quad,dblquad,simps,quad_vec,nquad

In [3]:
import tensorflow as tf

import keras
from keras import backend as K
from keras import datasets, layers, models
from keras.layers import Input, Dense, Flatten, Dropout, Conv2D, MaxPooling2D, Activation, concatenate
from keras.models import Sequential, save_model, load_model

from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Model
from keras import optimizers
from keras.callbacks import ModelCheckpoint

from matplotlib import pyplot
import pandas as pd
from glob import glob 
import csv

from sklearn.model_selection import train_test_split
from skimage.io import imread

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm
from colorspacious import cspace_converter
from collections import OrderedDict

import pylab as py
import threading
import scipy

from PIL import Image

from scipy import constants
from scipy import interpolate
from scipy import signal
from scipy import ndimage
from scipy.linalg import block_diag
import sympy

from matplotlib.image import imread
import skimage
from skimage.transform import rotate
from skimage.io import imshow, show, imsave
from skimage import img_as_ubyte


Using TensorFlow backend.


In [4]:
### Read the Input dataset -> Fourier Data which should look like reconstructed afterwards ###
input_df = pd.read_csv('data_cnn/labels.csv', index_col=0)
input_df['id'] = input_df.index.map(lambda id: id)
input_df['amplitude'] = input_df.index.map(lambda id: f'data_cnn/input_aug/{id}_ampl.png')
input_df['phase'] = input_df.index.map(lambda id: f'data_cnn/input_aug/{id}_phase.png')


In [5]:
def read_data(file_paths, img_rows, img_cols,  channels):
    """
      Reads the spectogram files from disk and normalizes the pixel values
        @params:
          file_paths - Array of file paths to read from
          img_rows - The image height.
          img_cols - The image width.
          as_grey - Read the image as Greyscale or RGB.
          channels - Number of channels.
        @returns:
          The created and compiled model (Model)        
    """
    images = []
  
    for file_path in file_paths:
        images.append(imread(file_path))
    
    images = np.asarray(images, dtype=np.float32)
  
  # normalize
    images = images / 255.0 #np.max(images)
  
  # reshape to match Keras expectaions
    images = images.reshape(images.shape[0], img_rows, img_cols, channels)

    return images

In [6]:
#### Hyper Parameters ####

## pixel if images ##
img_rows, img_cols = 120, 120
as_gray=True
in_channels = 3 # rgb files
num_classes = 3 # number of charts

batch_size = 32
epochs = 10
input_shape = (img_rows, img_cols, in_channels)
input_img = Input(shape= input_shape)

In [7]:
## read the images from CSV File ####

x_train_amplitude=read_data(input_df.amplitude.values, img_rows, img_cols, in_channels)
x_train_phase = read_data(input_df.phase.values, img_rows, img_cols, in_channels)

# labels - convert class vectors to binary class matrices One Hot Encoding
labels = input_df.appliance.values
labels = keras.utils.to_categorical(labels, num_classes)

np.shape(x_train_amplitude[0])

(120, 120, 3)

In [8]:
# split the train data for test batches

x_train_comp = np.stack((x_train_amplitude, x_train_phase), axis=4)

x_train, x_test, y_train, y_test = train_test_split(x_train_comp, labels, test_size=0.25, random_state=666)

# take them apart
x_train_amplitude = x_train[:,:,:,:,0]
x_test_amplitude = x_test[:,:,:,:,0]

x_train_phase = x_train[:,:,:,:,1]
x_test_phase = x_test[:,:,:,:,1]

print(np.shape(x_train_phase))

(36, 120, 120, 3)


In [9]:
x_test_phase.shape

(12, 120, 120, 3)

In [10]:
# X_train, X_test, y_train, y_test = train_test_split(x_train_amplitude, labels, test_size=0.2)

In [11]:
def create_convolution_layers(input_img):
    
    model = Conv2D(32, (3, 3), padding='same', input_shape=input_shape)(input_img)
    model = LeakyReLU(alpha=0.1)(model)
    model = MaxPooling2D((2, 2),padding='same')(model)
    model = Dropout(0.25)(model)
  
    model = Conv2D(64, (3, 3), padding='same')(model)
    model = LeakyReLU(alpha=0.1)(model)
    model = MaxPooling2D(pool_size=(2, 2),padding='same')(model)
    model = Dropout(0.25)(model)
    
    model = Conv2D(128, (3, 3), padding='same')(model)
    model = LeakyReLU(alpha=0.1)(model)
    model = MaxPooling2D(pool_size=(2, 2),padding='same')(model)
    model = Dropout(0.4)(model)


    return model

In [12]:
ampl_input = Input(shape=input_shape)
ampl_model = create_convolution_layers(ampl_input)

phase_input = Input(shape=input_shape)
phase_model = create_convolution_layers(phase_input)

conv = concatenate([ampl_model, phase_model])

conv = Flatten()(conv)

dense = Dense(512)(conv)
dense = LeakyReLU(alpha=0.1)(dense)
dense = Dropout(0.5)(dense)

output = Dense(num_classes, activation='softmax')(dense)

model = Model(inputs=[ampl_input, phase_input], outputs=[output])

opt = optimizers.Adam()

model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
model.summary()
# np.shape(conv)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 120, 120, 3)  0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 120, 120, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 120, 120, 32) 896         input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, 120, 120, 32) 896         input_3[0][0]                    
__________________________________________________________________________________________________
leaky_re_l

In [ ]:
### Train the model###

best_weights_file="weights.best.hdf5"
checkpoint = ModelCheckpoint(best_weights_file, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

callbacks = [checkpoint]


history=model.fit([x_train_amplitude, x_train_phase], y_train,
         batch_size=batch_size,
         epochs=epochs,
         callbacks=callbacks,
         verbose=1,
         validation_data=([x_test_amplitude, x_test_phase] ,y_test)
         ,shuffle=True
         )

In [ ]:
plt.plot(history.history['loss'], label='accuracy')
plt.plot(history.history['val_acc'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.4, 1])
plt.legend(loc='lower right')


In [ ]:
# Save the model
filepath = './saved_model'
# save_model(model, filepath)

In [ ]:
# Load the model
model = load_model(filepath, compile = True)

In [ ]:
# model.summary()
preds = model.evaluate([x_test_amplitude,x_test_phase],y_test)

print ('Loss = ' + str(preds[0]))
print ('Test Accuracy = ' + str(preds[1]))

In [ ]:
model.predict({'input_2':img_amp, 'input_3':img_ph})